In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_name = "dashing-shadow-171"
run_name = "vocal-sweep-28"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
       #idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
       #high_contradiction = get_idx_with_top_column_values('contradiction_scores',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i+1, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['label'])[0] )
    pp_all = list(df1['pp'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp","reward_pp","reward_pp_minus_baseline", "vm_scores","acceptability_scores","sts_scores",
                "contradiction_scores", "pp_letter_diff",'pp_logp','ref_logp','kl_div', "reward_penalty"]
                #,
                
    #g_fields = ["pp","vm_scores"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward_pp', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats_training_step(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

def show_random_examples_for_eval_set(df, n): 
    random_idx = df.idx.drop_duplicates().sample(n).tolist()
    cols = ['epoch','pp', 'is_adv_example', 'is_valid_pp', 'reward_pp', "vm_scores", 'acceptability_scores',
             'sts_scores', 'contradiction_scores',  'pp_letter_diff']#
         #   ,] #'edit_distance_token_level'
    
    metric_cols = [o for o in cols if o not in ['pp', 'epoch']]
    agg_d = {k:np.mean for k in metric_cols}
    for i, idx in enumerate(random_idx): 
        print("\n###############\n")
        print("random", i+1, "\n")
        df1 = df.query("idx==@idx")
        print("Original:", pd.unique(df1['orig'])[0])
        print("Original label", pd.unique(df1['label'])[0] )
        df2 = df1[cols]
        print("Paraphrase-level view")
        df3 = df2.groupby('pp').agg({'epoch': lambda x: list(x), **agg_d})
        df3.sort_values(by ='epoch', key=lambda col: col.map(lambda x: np.min(x)), inplace=True)
        display_all(df3)
        print("Epoch-level view")
        df4 = df2.groupby('epoch').agg(agg_d)
        df4.columns = [o + "_avg" for o in df4.columns]
        display_all(df4)
    #     print("Whole table")
    #     display_all(df2)
    return 


In [ ]:
split = 'training_step'
n=3

df_d = get_training_dfs(path_run, postprocessed=False)
df = df_d[split]
if split == "training_step":
    idx_d = get_interesting_idx(df, n)
    print_interesting_text_stats_training_step(df, n)
else: 
    show_random_examples_for_eval_set(df, n)


###############

random 1 

Original: It will focus on improving its profitability next year by streamlining operations .
Original label 2
Unique paraphrases: 76
How the paraphrases change:


,,,,,,,,,,,,epoch
pp,reward_pp,reward_pp_minus_baseline,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff,pp_logp,ref_logp,kl_div,reward_penalty,
She has focused its improvement on profitability next year by streamlining operations.,0.000000,0.000000,-5.364418e-06,0.955226,0.857132,0.051613,-3,-1.079995,-1.656895,0.267474,-0.106989,[1]
The company will focus on improving its profitability next year by completing a new strategy.,0.002597,0.000000,7.420778e-05,0.980280,0.839006,0.013392,-10,-1.042309,-0.986080,0.345496,-0.138198,[2]
It will be planning its further expansion in the run up to the following year.,0.000000,0.000000,9.995573e-01,0.960242,0.416356,0.064054,5,-1.986907,-2.919032,0.439015,-0.175606,[3]
Apple plans to boost profits by streamlining the business next year.,0.000000,0.000000,9.179115e-06,0.927224,0.569452,0.075544,15,-1.201447,-2.038569,0.518386,-0.207354,[4]
It's going to improve its profitability by simplifying its operations.,0.000000,0.000000,4.327297e-05,0.890745,0.777481,0.004148,13,-1.018146,-1.710427,0.676894,-0.270758,[5]
It's aiming to increase profitability by running the current operation.,0.000000,0.000000,-1.788139e-06,0.933104,0.746123,0.050636,12,-1.475245,-2.052383,0.487279,-0.194912,[6]
It will concentrate on improving its profitability next year through the flow of operations.,0.001147,0.000000,3.278255e-05,0.948423,0.885835,0.003678,-9,-0.721220,-1.086132,0.483406,-0.193362,[7]
The company wants to improve their profitability this year by simplifying operations.,0.000000,0.000000,1.871586e-05,0.961502,0.776850,0.118291,-2,-1.148046,-1.636380,0.491505,-0.196602,[8]
It will focus on profits next year by the operation of efficiency.,0.000000,0.000000,9.984629e-01,0.836258,0.684937,0.085629,17,-1.306366,-2.335228,0.441417,-0.176567,[9]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_lcp_conditions,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
31671,3754,24,It will focus on improving its profitability next year by streamlining operations .,It will concentrate on its profitability next year and the streamlining operations.,0.999675,0.015015,0.984804,2,1,1,0.984661,0.845175,0,1.0,0.006231,0.909375,-0.700732,-1.871568,1.170837,0.921895,10.0,9.008813,-0.368758,8.640055,6.054359,31,997,1,28.277012,0.441828,0.34375,24,32,20,32,0.290017,0.613721,0.012291,0.003312,0.004914,0.568713,0.000033,0.032327,0.004755,0.009781,0.000035,0.007199,0.00697,0.000538,0.289141,0.026439,0.051829,0.02565



###############

random 2 

Original: Renesas Mobile Europe Ltd has approximately 470 employees in Oulu .
Original label 1
Unique paraphrases: 55
How the paraphrases change:


...

,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_lcp_conditions,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
106096,3137,80,"The money will be spread mainly over 2011 and 2012 , the company said .",The money will rise to primarily being spread to 2011 and 2012.,0.999895,0.000408,0.999418,1,2,1,0.999487,0.86306,8,0.887324,0.010325,0.891732,-0.5493,-2.836942,2.287642,1.883758,10.0,1.421286,-0.753503,0.667782,0.366813,24,3342,0,1.01749,0.015898,0.875,24,32,23,32,0.336725,0.724233,0.013596,0.003745,0.005392,0.677763,0.00003,0.032498,0.004738,0.009858,0.000034,0.007414,0.006906,0.000524,0.337696,0.026277,0.000004,0.0



###############

label_flips 3 

Original: GET MIDNIGHTTRADER IN REALTIME : This report is delayed .
Original label 1
Unique paraphrases: 74
How the paraphrases change:


,,,,,,,,,,,,epoch
pp,reward_pp,reward_pp_minus_baseline,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff,pp_logp,ref_logp,kl_div,reward_penalty,
GET MIDNIGHTTRADER IN REALTIME : This report is delayed.,0.000859,0.000688,2.455711e-05,0.503965,1.000000,0.005852,1,-0.177522,-0.352835,0.344473,-0.137789,[1]
DOWNLOAD MIDNIGHTTRADER LIFESTYLE : This is delayed.,0.000000,0.000000,3.683567e-05,0.309430,0.729255,0.194469,5,-1.059952,-1.505600,0.310256,-0.124103,[2]
Take a look at midnight.,0.000000,0.000000,2.574921e-05,0.682096,0.506397,0.036144,33,-1.905286,-2.179329,0.484014,-0.193606,[3]
GET THE MINIDRIAM IN REAL MENU : This report is delayed.,0.000000,0.000000,1.287460e-05,0.538161,0.622615,0.097252,1,-1.386788,-2.333082,0.583517,-0.233407,[4]
IM TIDY in the real time : This report is delayed.,0.000000,0.000000,4.410744e-06,0.098973,0.588790,0.076203,7,-1.360514,-2.025030,0.697145,-0.278858,[5]
Get me Morningtrack in real time: This report is delayed.,0.000000,0.000000,3.755093e-05,0.519067,0.790503,0.766572,0,-1.085953,-1.635020,0.700328,-0.280131,[6]
You should be Minus of The Night to a real time.,0.000000,0.000000,5.960464e-07,0.468537,0.362853,0.339465,9,-2.410953,-3.702622,0.572817,-0.229127,[7]
MIDNIGHTTRADER in real time this report is delayed.,0.000955,0.000604,2.729893e-05,0.715928,0.944742,0.007670,6,-0.654051,-1.053747,0.579655,-0.231862,[8]
Get a meteorite in the REALTIME : This report is delayed,0.000000,0.000000,3.576279e-06,0.651864,0.503957,0.469450,1,-1.028871,-1.551034,0.676164,-0.270466,[9]


Best Paraphrase


,idx,epoch,orig,pp,orig_truelabel_probs,pp_truelabel_probs,pp_predclass_probs,label,pp_predclass,label_flip,vm_scores,sts_scores,pp_letter_diff,pp_letter_percent,contradiction_scores,acceptability_scores,pp_logp,ref_logp,diff_logp,kl_div,reward_pp,reward_pp_minus_baseline,reward_penalty,reward_pp_minus_baseline_with_penalty,loss,batch_num,global_step,acc_num,loss_sum,loss_batch,label_flip_fraction,orig_length,orig_batch_size,pp_length,pp_batch_size,time_generate_pp,time_loss_fn,time_pp_logp,time_log_entropy,time_log_token_probabilities,time_ref_logprobs,time_loss_fn_loss_calc,time_reward_fn,time_vm_scores,time_sts_scores,time_pp_letter_diff,time_contradiction_scores,time_acceptability_scores,time_lcp_conditions,time_backwards,time_calc_gradient_norm,time_opt_step_and_calc_param_norm,time_opt_step
106040,1808,80,GET MIDNIGHTTRADER IN REALTIME : This report is delayed .,Get MIDNIGHTTRADER IN REALTIME. This report has grown to delay.,0.999877,0.01316,0.986114,1,0,1,0.986718,0.932053,-6,1.105263,0.107428,0.626807,-0.108984,-2.040636,1.931652,1.685051,10.0,0.0,-0.674021,0.0,0.0,22,3340,0,0.979963,0.015312,0.78125,32,32,24,32,0.361869,0.909082,0.014191,0.003916,0.005599,0.860551,0.000026,0.03393,0.004938,0.010306,0.000036,0.007481,0.00722,0.000562,0.360038,0.034736,0.000005,0.0


In [ ]:


import string
linking_contrast_phrases = [o.strip() for o in open("./linking_contrast_phrases.txt").readlines()]
def check_for_linking_contrast(s):
    return any([s.startswith(phrase + " ") or s.endswith(" " + phrase) for phrase in linking_contrast_phrases])

sens_cleaned = [s.strip(string.punctuation).strip().lower() for s in sens]
[check_for_linking_contrast(s) for s in sens_cleaned ]


[True, True, True, True, True, True, True, True, True, False]

In [ ]:
%%timeit
sens_cleaned = [s.strip(string.punctuation).strip().lower() for s in df.orig.drop_duplicates().tolist()]
for s in sens_cleaned: 
    if check_for_linking_contrast(s): pass#print(s)

17.1 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepadre_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

not                   7
this                  5
love                  3
film                  2
movie                 2
apple                 2
like                  1
hate                  1
I do not like this    1
this apple            1
do                    1
I do not like         1
I love                1
dtype: int64

#### INSERTIONS ####

.                                    21
n't                                   7
like                                  2
that                                  2
film.                                 2
- I like it                           1
thyme                                 1
really                                1
Not so much                           1
'.                                    1
:                                     1
''.                                   1
love                                  1
ie                                    1
cherry.                               1
enjoy                          

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

ValueError: a must be greater than 0 unless no samples are taken